In [ ]:
import numpy as np
import pandas as pd
import random
import time
import warnings
from itertools import product
from scipy.stats import norm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Lasso, Ridge

from sklearn import linear_model, svm, naive_bayes, ensemble
from sklearn.model_selection import cross_validate, train_test_split, RepeatedStratifiedKFold
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix, roc_auc_score, matthews_corrcoef

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


## K-means functions

In [ ]:
def bhattacharyya_distance(p, q, n):
    PQ = np.sqrt(p * q)
    PQC = np.sqrt((1 - p) * (1 - q))
    return -n * np.log((PQ / PQC) + 1) - n * np.log(PQC)

def silhouette_score_new(data, cluster_labels):
  num_samples = len(data)

  # Initialize arrays to store a(i) and b(i) values for each sample
  a_values = np.zeros(num_samples)
  b_values = np.zeros(num_samples)

  # Iterate over each data point
  for i in range(num_samples):
      cluster_i = cluster_labels[i]

      # Calculate a(i) (average distance within the same cluster)
      indices_same_cluster = np.where(cluster_labels == cluster_i)[0]
      indices_same_cluster = indices_same_cluster[indices_same_cluster != i]  # Exclude the current point
      if len(indices_same_cluster) > 0:
        a_values[i] = np.mean([bhattacharyya_distance(data[i], data[j], n_patient_per_plan) for j in indices_same_cluster])
      else:
        a_values[i] = 0  # Set to 0 for singleton clusters

      # Calculate b(i) (smallest average distance to a different cluster)
      unique_clusters = np.unique(cluster_labels)
      b_values_i = np.zeros(len(unique_clusters))

      for j, cluster_j in enumerate(unique_clusters):
          if cluster_j != cluster_i:
              indices_other_cluster = np.where(cluster_labels == cluster_j)[0]
              b_values_i[j] = np.mean([bhattacharyya_distance(data[i], data[k], n_patient_per_plan) for k in indices_other_cluster])
          else:
              b_values_i[j] = np.inf

      b_values[i] = b_values_i.min()

  # Calculate silhouette scores for each data point
  silhouette_scores = (b_values - a_values) / np.maximum(a_values, b_values)

  # Calculate the average silhouette score
  average_silhouette_score = np.mean(silhouette_scores)

  # print("Silhouette Scores for each data point:")
  # print(silhouette_scores)
  # print("\nAverage Silhouette Score:", average_silhouette_score)

  return average_silhouette_score

def kmeans_bhattacharyya(data, k, max_iter = 100, seed=42):
  X = {'predicted_response_rate':data}
  X = pd.DataFrame(X)
  X_orig = X.copy()

#   print(X_orig)

  K = k

  # Randomly initialize centroids
  Centroids_orig = X_orig.sample(K, random_state=seed)
  custom_index = ['Distance_from_' + str(n) for n in range(1,k+1)]
  Centroids_orig = Centroids_orig.set_index(pd.Index(custom_index))
#   Centroids_orig
#   print("Centroids:", Centroids)

  diff = 1
  j = 0

  while diff != 0 and j <= max_iter:
    XD = X_orig.copy()
    Centroids = Centroids_orig.copy()

    for i in range(K):
        centroid_values = Centroids.iloc[i].values[0]

        distances = []
        for data_point in X['predicted_response_rate'].values:
            distance_result = bhattacharyya_distance(data_point, centroid_values, n = n_patient_per_plan)
            distances.append(distance_result)

        XD['Distance_from_{}'.format(i+1)] = distances

    # Assign each point to the closest cluster
    X['Cluster'] = XD.iloc[:, 1:K+1].idxmin(axis=1)
#     print(X)

    # Calculate new centroids
    Centroids_new = X.groupby(['Cluster']).mean()
#     print("Centroids_new:", Centroids_new)

    # Calculate the difference between old and new centroids
    if j == 0:
        diff = 1
        j += 1
    else:
#         print(K, Centroids_new.shape[0], Centroids.shape[0])
        Centroids = Centroids_orig.loc[Centroids_new.index]
        diff = np.abs(Centroids_new.values - Centroids.values).sum()
#         print(diff)
#         print(K, Centroids_new.shape[0], Centroids.shape[0])

    # Update centroids
    Centroids = Centroids_new
    j += 1


  X['cluster_number'] = X['Cluster'].str.extract(r'(\d+)').astype(int)

  results = {'clusters': X,
                  'centroids': Centroids}

  return results


In [ ]:
def kmeans_fit(data, max_iter=100):
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    x_train, x_test = train_test_split(x, test_size=0.2, random_state=42)
    length = min(len(x_train), 10)
    k_values = range(2, length)
#     print(k_values)
    # Initialize a list to store silhouette scores for different values of k
    silhouette_scores = []

    # Iterate over different values of k
    for k in k_values:
        # Fit the KMeans model to the training data
        kmeans_results = kmeans_bhattacharyya(data=x_train, k=k, max_iter = 10)
        x_test_tmp = pd.DataFrame(x_test, columns=['predicted_response_rate'])
        k_range = np.sort(kmeans_results['clusters']['cluster_number'].unique())
        for i in range(len(kmeans_results['centroids'])):
#             print(k, i)
            centroid_values = kmeans_results['centroids']['predicted_response_rate'].values[i]

            distances = []
            for data_point in x_test_tmp['predicted_response_rate']:
                distance_result = bhattacharyya_distance(data_point, centroid_values, n=n_patient_per_plan)
                distances.append(distance_result)

            x_test_tmp['Distance_from_{}'.format(k_range[i])] = distances

        x_test_tmp['Cluster'] = x_test_tmp.iloc[:, 1:k+1].idxmin(axis=1)
        x_test_tmp['cluster_number'] = x_test_tmp['Cluster'].str.extract(r'(\d+)').astype(int)

        silhouette_avg = silhouette_score_new(x_test_tmp['predicted_response_rate'], x_test_tmp['cluster_number'])
        silhouette_scores.append(silhouette_avg)

    # Find the k value with the highest silhouette score
#     print(silhouette_scores)
    best_k = k_values[np.argmax(silhouette_scores)]

    return {'best_k': best_k, 'silhouette_score': silhouette_scores}

In [ ]:
# def kmeans_fit(data, max_iter=10, n_rep = 10):
#     warnings.filterwarnings("ignore", category=RuntimeWarning)
#     random.seed(42)
#     random_seeds = [random.randint(1, 100000) for _ in range(n_rep)]

#     df_scores = pd.DataFrame()

#     for seed in random_seeds:
#         x_train, x_test = train_test_split(data, test_size=0.2, random_state=seed)
#         length = min(len(x_train), 10)
#         k_values = range(2, length)

#         # Initialize a list to store silhouette scores for different values of k
#         silhouette_scores = []

#         # Iterate over different values of k
#         for k in k_values:
#             # Fit the KMeans model to the training data
#             # print(k)
#             kmeans_results = kmeans_bhattacharyya(data=x_train, k=k, max_iter = max_iter)
#             x_test_tmp = pd.DataFrame(x_test, columns=['predicted_response_rate'])
#             for i in range(k):
#                 centroid_values = kmeans_results['centroids']['predicted_response_rate'].values[i]

#                 distances = []
#                 for data_point in x_test_tmp['predicted_response_rate']:
#                     distance_result = bhattacharyya_distance(data_point, centroid_values, n=n_patient_per_plan)
#                     distances.append(distance_result)

#                 x_test_tmp['Distance_from_{}'.format(i + 1)] = distances

#             x_test_tmp['Cluster'] = x_test_tmp.iloc[:, 1:k+1].idxmin(axis=1)
#             x_test_tmp['cluster_number'] = x_test_tmp['Cluster'].str.extract(r'(\d+)').astype(int)

#             silhouette_avg = silhouette_score_new(x_test_tmp['predicted_response_rate'], x_test_tmp['cluster_number'])
#             silhouette_scores.append(silhouette_avg)

# #         df_scores = df_scores.concat(pd.Series(silhouette_scores), ignore_index=True)
#         df_scores = pd.concat([df_scores, pd.Series(silhouette_scores)], ignore_index=True)

#     score_means = df_scores.mean()
#     # Find the k value with the highest silhouette score
#     best_k = k_values[np.argmax(score_means)]

#     return {'best_k': best_k, 'silhouette_score': score_means}

## Ensemble model

In [ ]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0)),
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
#                     ('RF', ensemble.RandomForestClassifier(n_estimators=200, criterion='gini', random_state=0))
                    ('XGB', XGBClassifier(n_estimators=50, learning_rate=0.1, random_state=0))
                   ],
        voting='hard'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        'LR__C': [0.01, 0.5, 1.0], # [0.01],
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.01, 0.05, 0.1]
#         'RF__n_estimators': [50] # [10, 30, 50]
        'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

In [ ]:
# from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
# from xgboost import XGBClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

In [ ]:
# def ensemble_model_fit(data, data_pred):
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan','plan_response_rate','group_size'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the VotingClassifier with the individual classifiers
#     voting_classifier = VotingClassifier(
#         estimators=[
#             ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0)),
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0)),
#             ('Lasso', linear_model.LogisticRegression(penalty='l1', solver='liblinear', max_iter=200, random_state=0)),
#             ('RF', RandomForestClassifier(n_estimators=200, criterion='gini', random_state=0)),
#             ('XGB', XGBClassifier(n_estimators=200, learning_rate=1.0, random_state=0)),
#             ('GBM', GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=0)),
#             ('NN', MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=200, random_state=0))
#         ],
#         voting='soft'
#     )

#     # Define the hyperparameter grid to search
#     param_grid = {
#         'LR__C': [0.01],
#         'Ridge__C': [0.01],
#         'RF__n_estimators': [50],
#         'XGB__n_estimators': [50],
#         'GBM__n_estimators': [50],
#         'NN__hidden_layer_sizes': [(100,)],
#     }


#     # Create a GridSearchCV object
#     # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
#     grid_search = GridSearchCV(voting_classifier, param_grid, cv=5, scoring='roc_auc')

#     # Perform the grid search on the training data
#     grid_search.fit(X_train, y_train)

#     # Get the best hyperparameters
#     best_params = grid_search.best_params_

#     # Train the final VotingClassifier with the best hyperparameters on the full training set
#     final_voting_classifier = grid_search.best_estimator_
#     final_voting_classifier.fit(X_train, y_train)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
#     y_pred_test = final_voting_classifier.predict(X_test)
#     X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size'], axis=1)
#     y_pred = final_voting_classifier.predict_proba(X_dt)

#     return y_pred

## Sample size determination

In [ ]:
def sample_size_calc(orr_1, n_1, delta, alpha, power):
    orr_2 = orr_1 + delta
    numerator = orr_2 * (1-orr_2)

    denominator1 = (delta/(norm.ppf(1 - alpha) + norm.ppf(power)))**2
    denominator2 = (orr_1 * (1-orr_1))/n_1

#     print(numerator, denominator1, denominator2)

    return numerator/(denominator1 - denominator2)

## Data generations for each steps

In [ ]:
design_list = [5,8,10]
patient_n_list = [5400,680,170] # n_patient_per_plan

def generate_data_step1(design_number = 5, n_rounds = 4, n_patient_per_plan = 5400, seed=42):

  # design_list = [5,8,10]

  # Record the start time
#   start_time = time.time()

  random.seed(seed)

  n_design = design_number
  n_patient_per_plan_round = n_patient_per_plan/n_rounds # each round is capped by this patient number

#   total_n = n_patient_per_plan * (2 ** n_design)

  # Create all possible combinations of design features
  design_combinations = list(product([1, 0], repeat=n_design))

  # Create a DataFrame from the combinations
  design_feature_df = pd.DataFrame(design_combinations, columns=[f'Design_Feature_{i+1}' for i in range(n_design)])
  design_feature_df['recruitment_plan'] = [x + 1 for x in range(2 ** n_design)]

  # scenario 1:
  random.seed(seed)
  plan_response_rate = [random.uniform(0.01, 0.1) for _ in range(2**n_design)]
  design_feature_df['plan_response_rate'] = plan_response_rate

  # each combination repeat for n_patient_per_plan_round
  design_feature_df = pd.DataFrame(np.repeat(design_feature_df.values, n_patient_per_plan_round, axis=0), columns=design_feature_df.columns)

  # add response outcome column:
  grouped_df = design_feature_df.groupby(list(design_feature_df.columns)).size().reset_index(name='group_size')

  def generate_responses(row):
      rate = row['plan_response_rate']
      num = row['group_size']
      # random.seed(42)
      np.random.seed(seed)
      return np.random.binomial(n=1, p=rate, size=int(num))

  grouped_df['response'] = grouped_df.apply(generate_responses, axis=1)

  # Explode the 'response' arrays to expand the DataFrame
  expanded_df = grouped_df.explode('response').reset_index(drop=True)
  expanded_df['response'] = expanded_df['response'].astype(float)

#   end_time = time.time()

#   elapsed_time = end_time - start_time

#   print(f"Elapsed time: {elapsed_time} seconds")

  return expanded_df

In [ ]:
design_list = [5,8,10]
patient_n_list = [5400,680,170]

def generate_data_step2up(highest_cluster, p_vec, design_number = 5, n_rounds = 4, n_patient_per_plan = 5400, size = 5400, seed=42):

  random.seed(seed)

  n_design = design_number
  n_patient_per_plan_round = n_patient_per_plan/n_rounds # each round is capped by this patient number

  # Use numpy.random.choice to sample rows with replacement
#   selected_rows_indices = np.random.choice(highest_cluster.index, size=int(n_patient_per_plan_round*(2**n_design)), p=p_vec)
  selected_rows_indices = np.random.choice(highest_cluster.index, size=size, p=p_vec) # total data size = size
  sampled_rows_step2 = highest_cluster.iloc[selected_rows_indices]

  # add response outcome column:
  grouped_df = sampled_rows_step2.groupby(list(sampled_rows_step2.columns)).size().reset_index(name='group_size')

  def generate_responses(row):
      rate = row['plan_response_rate']
      num = row['group_size']
      np.random.seed(seed)
      return np.random.binomial(n=1, p=rate, size=int(num))

  grouped_df['response'] = grouped_df.apply(generate_responses, axis=1)

  # Explode the 'response' arrays to expand the DataFrame
  expanded_df = grouped_df.explode('response').reset_index(drop=True)
  expanded_df['response'] = expanded_df['response'].astype(float)


  return expanded_df

In [ ]:
def generate_data_step3(design_number = 5):

  random.seed(42)
  n_design = design_number
  n_patient_per_plan_round = n_patient_per_plan/n_rounds # each round is capped by this patient number

  # Use numpy.random.choice to sample rows with replacement
  selected_rows_indices = np.random.choice(highest_cluster_step3.index, size=n_patient_per_plan*(2**n_design), p=p_vec_step3)

  sampled_rows_step3 = highest_cluster_step3.iloc[selected_rows_indices]

  # add response outcome column:
  grouped_df = sampled_rows_step3.groupby(list(sampled_rows_step3.columns)).size().reset_index(name='group_size')

  def generate_responses(row):
      rate = row['plan_response_rate']
      num = row['group_size']
      np.random.seed(44)
      return np.random.binomial(n=1, p=rate, size=int(num))

  grouped_df['response'] = grouped_df.apply(generate_responses, axis=1)

  # Explode the 'response' arrays to expand the DataFrame
  expanded_df = grouped_df.explode('response').reset_index(drop=True)
  expanded_df['response'] = expanded_df['response'].astype(float)

  return expanded_df